- Extraction of Sentence

In [ ]:
import pandas as pd
import Sentiment
import sqlite3
# 신조어 목록 불러오기
df = pd.read_excel('new_words_var.xlsx')
new_words = df[df.y == 1].iloc[:,1].drop_duplicates(keep='first')

# 전체 데이터 합치기
conn = sqlite3.connect('Total_Ent.db')
df1 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
conn = sqlite3.connect('HP.db')
df2 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
head = df1.append(df2)

# sentence 추출
analysis = Sentiment.Sentiment()
sent = analysis.extract_sent(head, new_words)
sent = pd.DataFrame.from_dict(sent, orient='index')
conn = sqlite3.connect('sent.db')
sent.to_sql('sent', conn)

- Update Sentiment Dictionary and Extraction of Sentiment Score

In [ ]:
import Sentiment
from tqdm.notebook import tqdm
import sqlite3
import pandas as pd

# Initialize
analysis = Sentiment.Sentiment()
conn = sqlite3.connect('sent.db')
sent = pd.read_sql('SELECT * FROM sent', conn)
mapping_most_related = analysis.cal_score(sent)

positive = []
negative = []
for _ in tqdm(range(10)):
    # 일정 (단어-감성score) set을 사용하여, 감성사전 score 업데이트 (학습과정)
    sentiment_result = pd.read_excel('sentiment_result.xlsx')
    positive, negative = analysis.update_score(positive, negative, sentiment_result) # 감성사전 score 업데이트
    
    # 감성사전을 이용하여 감성분석을 진행
    sentiment = pd.read_excel('sentiment.xlsx')
    analysis.sentiment_analysis(sentiment, mapping_most_related)

- Extract of keywords

In [18]:
import Sentiment
from tqdm.notebook import tqdm
import sqlite3
import pandas as pd

# Initialize
analysis = Sentiment.Sentiment()
conn = sqlite3.connect('News_sent.db')
sent = pd.read_sql('SELECT * FROM sent', conn)

In [19]:
# keyword 추출
from collections import defaultdict
length = list(range(len(sent)))
length.pop(-1)
ktemp = defaultdict(lambda:0)
rtemp = defaultdict(lambda:0)
for _ in length:
    words = {sent.iloc[_]['index']:1.0}
    keywords = analysis.most_similar(sent.iloc[_]['index'], sent.iloc[_]['0'], words)
    ktemp[sent.iloc[_]['index']] = keywords[0]
    rtemp[sent.iloc[_]['index']] = keywords[1]
pd.DataFrame.from_dict(ktemp, orient='index').to_excel('keyword.xlsx')
pd.DataFrame.from_dict(rtemp, orient='index').to_excel('rkeyword.xlsx')

training was done. used memory 0.310 Gb0.310 Gb
all cohesion probabilities was computed. # words = 208
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.310 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.310 Gb
  - (word, context) matrix was constructed. shape = (221, 221)                    
  - done
training was done. used memory 0.310 Gb0.310 Gb
all cohesion probabilities was computed. # words = 216
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.310 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.310 Gb
  - (word, context) matrix was constructed. shape = (17, 17)                    
  - done
training was done. used memory 0.316 Gb0.316 Gb
all cohesion probabilities was computed. # words = 820
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.316 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.316 Gb
  - (word, context) matrix was constructed. shape = (1030, 1030) 

training was done. used memory 0.316 Gb0.316 Gb
all cohesion probabilities was computed. # words = 1926
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.316 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.316 Gb
  - (word, context) matrix was constructed. shape = (102, 102)                    
  - done
training was done. used memory 0.316 Gb0.316 Gb
all cohesion probabilities was computed. # words = 1963
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.316 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.316 Gb
  - (word, context) matrix was constructed. shape = (120, 120)                    
  - done
training was done. used memory 0.316 Gb0.316 Gb
all cohesion probabilities was computed. # words = 2120
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.316 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.316 Gb
  - (word, context) matrix was constructed. shape = (405, 40

training was done. used memory 0.317 Gb0.317 Gb
all cohesion probabilities was computed. # words = 3353
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.317 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.317 Gb
  - (word, context) matrix was constructed. shape = (267, 267)                    
  - done
training was done. used memory 0.317 Gb0.317 Gb
all cohesion probabilities was computed. # words = 3357
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.317 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.317 Gb
  - (word, context) matrix was constructed. shape = (21, 21)                    
  - done
training was done. used memory 0.317 Gb0.317 Gb
all cohesion probabilities was computed. # words = 3422
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.317 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.317 Gb
  - (word, context) matrix was constructed. shape = (273, 273)

training was done. used memory 0.316 Gb0.319 Gb
all cohesion probabilities was computed. # words = 4428
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.316 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.316 Gb
  - (word, context) matrix was constructed. shape = (146, 146)                    
  - done
training was done. used memory 0.317 Gb0.316 Gb
all cohesion probabilities was computed. # words = 4432
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.317 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.317 Gb
  - (word, context) matrix was constructed. shape = (11, 11)                    
  - done
training was done. used memory 0.316 Gb0.318 Gb
all cohesion probabilities was computed. # words = 4442
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.316 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.316 Gb
  - (word, context) matrix was constructed. shape = (69, 69)  

training was done. used memory 0.319 Gb0.319 Gb
all cohesion probabilities was computed. # words = 5077
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.319 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.319 Gb
  - (word, context) matrix was constructed. shape = (162, 162)                    
  - done
training was done. used memory 0.319 Gb0.319 Gb
all cohesion probabilities was computed. # words = 5101
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.319 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.319 Gb
  - (word, context) matrix was constructed. shape = (20, 20)                    
  - done
training was done. used memory 0.319 Gb0.319 Gb
all cohesion probabilities was computed. # words = 5108
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.319 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.319 Gb
  - (word, context) matrix was constructed. shape = (12, 12)  

...

training was done. used memory 0.321 Gb0.321 Gb
all cohesion probabilities was computed. # words = 6880
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.321 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.321 Gb
  - (word, context) matrix was constructed. shape = (15, 15)                    
  - done
training was done. used memory 0.321 Gb0.321 Gb
all cohesion probabilities was computed. # words = 6919
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.321 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.321 Gb
  - (word, context) matrix was constructed. shape = (67, 67)                    
  - done
training was done. used memory 0.321 Gb0.321 Gb
all cohesion probabilities was computed. # words = 7041
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.321 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.321 Gb
  - (word, context) matrix was constructed. shape = (326, 326)  

training was done. used memory 0.322 Gb0.322 Gb
all cohesion probabilities was computed. # words = 7936
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.322 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.322 Gb
  - (word, context) matrix was constructed. shape = (92, 92)                    
  - done
training was done. used memory 0.322 Gb0.322 Gb
all cohesion probabilities was computed. # words = 7960
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.322 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.322 Gb
  - (word, context) matrix was constructed. shape = (83, 83)                    
  - done
training was done. used memory 0.322 Gb0.322 Gb
all cohesion probabilities was computed. # words = 7998
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.322 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.322 Gb
  - (word, context) matrix was constructed. shape = (115, 115)  

training was done. used memory 0.327 Gb0.327 Gb
all cohesion probabilities was computed. # words = 8926
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.327 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.327 Gb
  - (word, context) matrix was constructed. shape = (26, 26)                    
  - done
training was done. used memory 0.327 Gb0.327 Gb
all cohesion probabilities was computed. # words = 8929
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.327 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.327 Gb
  - (word, context) matrix was constructed. shape = (7, 7)                    
  - done
training was done. used memory 0.327 Gb0.327 Gb
all cohesion probabilities was computed. # words = 8931
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.327 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.327 Gb
  - (word, context) matrix was constructed. shape = (4, 4)        

training was done. used memory 0.327 Gb0.327 Gb
all cohesion probabilities was computed. # words = 9272
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.327 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.327 Gb
  - (word, context) matrix was constructed. shape = (5, 5)                    
  - done
training was done. used memory 0.327 Gb0.327 Gb
all cohesion probabilities was computed. # words = 9272
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.327 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.327 Gb
  - (word, context) matrix was constructed. shape = (18, 18)                    
  - done
training was done. used memory 0.327 Gb0.327 Gb
all cohesion probabilities was computed. # words = 9302
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.327 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.327 Gb
  - (word, context) matrix was constructed. shape = (57, 57)      

training was done. used memory 0.327 Gb0.327 Gb
all cohesion probabilities was computed. # words = 9791
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.327 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.327 Gb
  - (word, context) matrix was constructed. shape = (49, 49)                    
  - done
training was done. used memory 0.327 Gb0.327 Gb
all cohesion probabilities was computed. # words = 9797
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.327 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.327 Gb
  - (word, context) matrix was constructed. shape = (46, 46)                    
  - done
training was done. used memory 0.327 Gb0.327 Gb
all cohesion probabilities was computed. # words = 9814
Create (word, contexts) matrix
  - counting word frequency from 0 sents, mem=0.327 Gb
  - scanning (word, context) pairs from 0 sents, mem=0.327 Gb
  - (word, context) matrix was constructed. shape = (13, 13)    

- Top 5 keyword Extraction

In [20]:
import pandas as pd
import ast
keyword = pd.read_excel('keyword.xlsx')
rkeyword = pd.read_excel('rkeyword.xlsx')
df = pd.merge(keyword, rkeyword, on='Unnamed: 0')

In [21]:
from collections import defaultdict
key = defaultdict(lambda:0)
for i in range(len(df)):
    temp = [j for j in df.iloc[i,:] if type(j) == str]
    key[temp[0]] = temp[1:]
    
for k,v in key.items():
    temp = [ast.literal_eval(_) for _ in v]
    temp = sorted(temp, key=lambda _:_[-1], reverse=True)
    while len(temp) > 5:
        temp.pop(-1)
    key[k] = temp

pd.DataFrame.from_dict(key, orient='index').to_excel('News_key.xlsx')

- Convert sentiment score to percent

In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm

score = pd.read_excel('News_sentiment_result.xlsx') # 감성분석 결과 가져오기
# 정규화
temp = np.array(score[1])
temp = StandardScaler().fit_transform(temp.reshape(-1,1))
# 정규분포 누적확률 구하기
temp = norm.cdf(temp)*100
# 긍정, 부정으로 DataFrame에 집어넣기
score[1] = temp
score[2] = 100-score[1]
score.rename({0:'Word', 1:'Positive', 2:'Negative'}, axis=1, inplace=True)